# Pre-bootcamp exercises: developing analysis_tools package in a notebook

**Description:** developing LSST Science Pipelines packages in the Notebook Aspect (based on this [guide](https://nb.lsst.io/science-pipelines/development-tutorial.html)).

**Contact authors:** Keith Bechtol

**Last verified to run:** 2023-05-11

**LSST Science Piplines version:** w_2023_19

**Container Size:** Small (or larger)

**Location:** This notebook points to files on the S3DF cluster at the USDF. Update paths accordingly if you are running elsewhere.

**Skills:** 
- Science Pipelines package development and test in a notebook

## Setting up and building a local version of analysis_tools package

Check the version of the stack you are using

In [ ]:
!eups list -s | grep lsst_distrib

If you are doing development on the `analysis_tools` package and want to test in a notebook, follow the guidance [here](https://nb.lsst.io/science-pipelines/development-tutorial.html). Brief version below (for work on the RSP at USDF):

1. In the termal, clone the [analysis_tools](https://github.com/lsst/analysis_tools) repo and set up the package

```
source /opt/lsst/software/stack/loadLSST.bash
setup lsst_distrib

# Choose file location for your repo
cd ~/repos/
git clone https://github.com/lsst/analysis_tools.git
cd analysis_tools
setup -k -r .
scons
```

2. Add the following line to `~/notebooks/.user_setups`

```
setup -k -r ~/repos/analysis_tools
```

Your local version of `analysis_tools` should now be accessible in a notebook.

In [ ]:
import numpy as np

In [ ]:
import lsst.analysis.tools
print(lsst.analysis.tools.__file__)

## Add a new action to your local analysis_tools version

Add the following example `MultiplyByScalar` class definition to the file `python/lsst/analysis/tools/actions/vector/vectorActions.py` in your local `analysis_tools` repo:

```
class MultiplyByScalar(VectorAction):
    """Multiply vector by a scalar value"""

    vectorKey = Field[str](doc="Key of vector which should be loaded")
    factor = Field[float](doc="Multiplicative factor", default=1.)

    def getInputSchema(self) -> KeyedDataSchema:
        return ((self.vectorKey, Vector),)

    def __call__(self, data: KeyedData, **kwargs) -> Vector:
        return np.array(self.factor * data[self.vectorKey.format(**kwargs)])
```

Also need to add one line to the `__all__` definition for the newly defined class:

```
__all__ = (
    "DownselectVector",
    "MultiCriteriaDownselectVector",
    "MagColumnNanoJansky",
    "FractionalDifference",
    "Sn",
    "ConstantValue",
    "SubtractVector",
    "DivideVector",
    "LoadVector",
    "MagDiff",
    "SNCalculator",
    "ExtinctionCorrectedMagDiff",
    "PerGroupStatistic",
    "ResidualWithPerGroupStatistic",
    "RAcosDec",
    "ConvertUnits",
    "MultiplyByScalar", # This is the new line to add
)
```

Import your new action

In [ ]:
from lsst.analysis.tools.actions.vector import MultiplyByScalar

Create some synthetic data to use for testing

In [ ]:
# create some KeyedData
data = {"randomData": np.random.normal(0, 3, 10000)}

Run your newly defined action

In [ ]:
action = MultiplyByScalar(vectorKey="randomData", factor=2.)
results = action(data)
assert np.allclose(results, 2. * data['randomData'])